# LLM and RAG Evaluation

Sources: [1](https://docs.llamaindex.ai/en/stable/module_guides/evaluating/), [2](), [3](), [4](), [5]()  

LLMs are trained on enormous bodies of data but they aren’t trained on your data. Retrieval-Augmented Generation (RAG) solves this problem by adding your data to the data LLMs already have access to. You will see references to RAG frequently in this documentation.  
In RAG, your data is loaded and prepared for queries or “indexed”. User queries act on the index, which filters your data down to the most relevant context. This context and your query then go to the LLM along with a prompt, and the LLM provides a response.  
Even if what you’re building is a chatbot or an agent, you’ll want to know RAG techniques for getting data into your application.  

Evaluation and benchmarking are crucial concepts in LLM development. To improve the performance of an LLM app (RAG, agents), you must have a way to measure it.

LlamaIndex offers key modules to measure the quality of generated results. We also offer key modules to measure retrieval quality.

### 1.Response Evaluation:  
Does the response match the retrieved context? Does it also match the query? Does it match the reference answer or guidelines?

### 2. Retrieval Evaluation:  
Are the retrieved sources relevant to the query?

This section describes how the evaluation components within LlamaIndex work.

---  
## 1. Response Evaluation

Evaluation of generated results can be difficult, since unlike traditional machine learning the predicted result isn't a single number, and it can be hard to define quantitative metrics for this problem. LlamaIndex offers LLM-based evaluation modules to measure the quality of results. This uses a "gold" LLM (e.g. GPT-4) to decide whether the predicted answer is correct in a variety of ways. Note that many of these current evaluation modules do not require ground-truth labels. Evaluation can be done with some combination of the query, context, response, and combine these with LLM calls.

These evaluation modules are in the following forms:

+ #### Correctness: Whether the generated answer matches that of the reference answer given the query (requires labels).
+ #### Semantic Similarity Whether the predicted answer is semantically similar to the reference answer (requires labels).
+ #### Faithfulness: Evaluates if the answer is faithful to the retrieved contexts (in other words, whether if there's hallucination).
+ #### Context Relevancy: Whether retrieved context is relevant to the query.
+ #### Answer Relevancy: Whether the generated answer is relevant to the query.
+ #### Guideline Adherence: Whether the predicted answer adheres to specific guidelines.

+ Question Generation: In addition to evaluating queries, LlamaIndex can also use your data to generate questions to evaluate on. This means that you can automatically generate questions, and then run an evaluation pipeline to test if the LLM can actually answer questions accurately using your data.

---
## 2. Retrieval Evaluation

We also provide modules to help evaluate retrieval independently.

The concept of retrieval evaluation is not new; given a dataset of questions and ground-truth rankings, we can evaluate retrievers using ranking metrics like mean-reciprocal rank (MRR), hit-rate, precision, and more.

The core retrieval evaluation steps revolve around the following:

+ #### Dataset generation: Given an unstructured text corpus, synthetically generate (question, context) pairs.  
+ #### Retrieval Evaluation: Given a retriever and a set of questions, evaluate retrieved results using ranking metrics.  
--- 

#### Installing Packages

In [1]:
!pip install -q openai
!pip install -q llama-index
!pip install -q llama-index-experimental
!pip install -q llama-index-llms-openai

#### Importing Packages

In [2]:
import os
import openai

#os.environ["OPENAI_API_KEY"] = "<the key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

import sys
import shutil
import glob
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import pandas as pd


import llama_index

## Llamaindex readers
from llama_index.core import SimpleDirectoryReader

## LlamaIndex Index Types
from llama_index.core import ListIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import TreeIndex
from llama_index.core import KeywordTableIndex
from llama_index.core import SimpleKeywordTableIndex
from llama_index.core import DocumentSummaryIndex
from llama_index.core import KnowledgeGraphIndex
from llama_index.experimental.query_engine import PandasQueryEngine

## LlamaIndex Context Managers
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.schema import Node

## LlamaIndex Callbacks
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler

In [3]:
import logging

#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Defining Models

In [4]:
models = """gpt-4, gpt-4-32k, gpt-4-1106-preview, gpt-4-0125-preview, gpt-4-turbo-preview, 
gpt-4-vision-preview, gpt-4-1106-vision-preview, gpt-4-turbo-2024-04-09, gpt-4-turbo, gpt-4-0613, 
gpt-4-32k-0613, gpt-4-0314, gpt-4-32k-0314, gpt-3.5-turbo, gpt-3.5-turbo-16k, gpt-3.5-turbo-0125, 
gpt-3.5-turbo-1106, gpt-3.5-turbo-0613, gpt-3.5-turbo-16k-0613, gpt-3.5-turbo-0301, text-davinci-003, 
text-davinci-002, gpt-3.5-turbo-instruct, text-ada-001, text-babbage-001, text-curie-001, ada, babbage, 
curie, davinci, gpt-35-turbo-16k, gpt-35-turbo, gpt-35-turbo-0125, gpt-35-turbo-1106, gpt-35-turbo-0613, 
gpt-35-turbo-16k-0613""".split()
models = [m.strip(", ") for m in models]
models

['gpt-4',
 'gpt-4-32k',
 'gpt-4-1106-preview',
 'gpt-4-0125-preview',
 'gpt-4-turbo-preview',
 'gpt-4-vision-preview',
 'gpt-4-1106-vision-preview',
 'gpt-4-turbo-2024-04-09',
 'gpt-4-turbo',
 'gpt-4-0613',
 'gpt-4-32k-0613',
 'gpt-4-0314',
 'gpt-4-32k-0314',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-0125',
 'gpt-3.5-turbo-1106',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-0301',
 'text-davinci-003',
 'text-davinci-002',
 'gpt-3.5-turbo-instruct',
 'text-ada-001',
 'text-babbage-001',
 'text-curie-001',
 'ada',
 'babbage',
 'curie',
 'davinci',
 'gpt-35-turbo-16k',
 'gpt-35-turbo',
 'gpt-35-turbo-0125',
 'gpt-35-turbo-1106',
 'gpt-35-turbo-0613',
 'gpt-35-turbo-16k-0613']

In [6]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

#model="gpt-3.5-turbo"
model="gpt-3.5-turbo-0125"
#model="gpt-4"
#model="gpt-4-turbo"

Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
embed_model = Settings.embed_model
Settings.llm = OpenAI(temperature=0, model=model)
llm = Settings.llm

In [7]:
import nest_asyncio
nest_asyncio.apply()

## 1. Response Evaluation

### 1.1 Correctness  
The CorrectnessEvaluator evaluates the relevance and correctness of a generated answer against a reference answer.

In [8]:
from llama_index.core.evaluation import CorrectnessEvaluator
evaluator = CorrectnessEvaluator(llm=llm)

In [9]:
query = ("Can you explain the theory of relativity proposed by Albert Einstein in detail?")

reference = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, 
published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a 
vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).
General relativity, published in 1915, extended these ideas to include the effects of gravity. According to general relativity, gravity is not a force between 
masses, as described by Newton's theory of gravity, but rather the result of the warping of space and time by mass and energy. Massive objects, such as 
planets and stars, cause a curvature in spacetime, and smaller objects follow curved paths in response to this curvature. This concept is often illustrated 
using the analogy of a heavy ball placed on a rubber sheet, causing it to create a depression that other objects (representing smaller masses) naturally move 
towards.
In essence, general relativity provided a new understanding of gravity, explaining phenomena like the bending of light by gravity (gravitational lensing) and the precession of the orbit of Mercury. It has been confirmed through numerous experiments and observations and has become a fundamental theory in modern physics.
"""

response = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, 
published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a 
vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) 
and mass (m).
However, general relativity, published in 1915, extended these ideas to include the effects of magnetism. According to general relativity, 
gravity is not a force between masses but rather the result of the warping of space and time by magnetic fields generated by massive objects. 
Massive objects, such as planets and stars, create magnetic fields that cause a curvature in spacetime, and smaller objects follow curved paths 
in response to this magnetic curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet with magnets 
underneath, causing it to create a depression that other objects (representing smaller masses) naturally move towards due to magnetic attraction.
"""

In [10]:
result = evaluator.evaluate(query=query, response=response, reference=reference,)
print(result.score)
print(result.feedback)

None
4.0
The generated answer provides a detailed explanation of Albert Einstein's theory of relativity, covering both special relativity and general relativity. It correctly mentions the key concepts such as the constancy of the speed of light, the equation E=mc², and the warping of space and time by mass and energy. However, there is a mistake in stating that general relativity includes the effects of magnetism instead of gravity. This error impacts the overall correctness of the answer, but the relevance and coverage of the topic warrant a score of 4.


### 1.2 Semantic Similarity  
The SemanticSimilarityEvaluator evaluates the quality of a question answering system via semantic similarity.  
Concretely, it calculates the similarity score between embeddings of the generated answer and the reference answer.  

In [11]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator
evaluator = SemanticSimilarityEvaluator()

In [12]:
# This evaluator only uses `response` and `reference`, passing in query does not influence the evaluation
# query = 'What is the color of the sky'

response = "The sky is typically blue"
reference = """The color of the sky can vary depending on several factors, including time of day, weather conditions, and location.

During the day, when the sun is in the sky, the sky often appears blue. 
This is because of a phenomenon called Rayleigh scattering, where molecules and particles in the Earth's atmosphere scatter sunlight in all directions, and blue light is scattered more than other colors because it travels as shorter, smaller waves. 
This is why we perceive the sky as blue on a clear day.
"""

result = await evaluator.aevaluate(response=response, reference=reference,)
print("Score: ", result.score)
print("Passing: ", result.passing)  # default similarity threshold is 0.8

Score:  0.8741614884630503
Passing:  True


In [13]:
response = "Sorry, I do not have sufficient context to answer this question."
reference = """The color of the sky can vary depending on several factors, including time of day, weather conditions, and location.

During the day, when the sun is in the sky, the sky often appears blue. 
This is because of a phenomenon called Rayleigh scattering, where molecules and particles in the Earth's atmosphere scatter sunlight in all directions, and blue light is scattered more than other colors because it travels as shorter, smaller waves. 
This is why we perceive the sky as blue on a clear day.
"""

result = await evaluator.aevaluate(response=response, reference=reference,)
print("Score: ", result.score)
print("Passing: ", result.passing)  # default similarity threshold is 0.8

Score:  0.7213441101430746
Passing:  False


#### Customization

In [14]:
from llama_index.core.embeddings import resolve_embed_model
evaluator = SemanticSimilarityEvaluator(embed_model=embed_model, similarity_threshold=0.6,)

In [15]:
response = "The sky is yellow."
reference = "The sky is blue."

result = await evaluator.aevaluate(response=response, reference=reference,)
print("Score: ", result.score)
print("Passing: ", result.passing)

Score:  0.9406303029427779
Passing:  True


We note here that a high score does not imply the answer is always correct.   
Embedding similarity primarily captures the notion of "relevancy". Since both the response and reference discuss "the sky" and colors, they are semantically similar.

### 1.3 Faithfulness

The `FaithfulnessEvaluator` module measures if the response from a query engine matches any source nodes.  
This is useful for measuring if the response was hallucinated.  
The data is extracted from the [New York City](https://en.wikipedia.org/wiki/New_York_City) wikipedia page.

In [16]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Response,
)
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
pd.set_option("display.max_colwidth", 0)

In [31]:
evaluator = FaithfulnessEvaluator(llm=llm)

In [18]:
documents = SimpleDirectoryReader(input_files=["../Data/nyc_text.txt"]).load_data()
splitter = SentenceSplitter(chunk_size=512)
vector_index = VectorStoreIndex.from_documents(documents, transformations=[splitter])

In [19]:
from llama_index.core.evaluation import EvaluationResult

In [20]:
# define jupyter display function
def display_eval_df(response: Response, eval_result: EvaluationResult) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
            "Reasoning": eval_result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

To run evaluations you can call the `.evaluate_response()` function on the `Response` object return from the query to run the evaluations.  
Lets evaluate the outputs of the vector_index.

In [32]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query("How did New York City get its name?")
eval_result = evaluator.evaluate_response(response=response_vector)
display_eval_df(response_vector, eval_result)

,Response,Source,Evaluation Result,Reasoning
0,"New York City was named after King Charles II of England granted the lands to his brother, the Duke of York.","The city came under British control in 1664 and was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city was regained by the Dutch in July 1673 and was renamed New Orange for one year and three months; the city has been continuously named New York since November 1674. New York City was the capital of the United States from 1785 until 1790, and has been the largest U.S. city since 1790. The Statue of Liberty greeted millions of immigrants as they came to the U.S. by ship in the late 19th and early 20th centuries, and is a symbol of the U.S. and its ideals of liberty and peace. In the 21st century, New York City has emerged as a global node of creativity, entrepreneurship, and as a symbol of freedom and cultural diversity. The New York Times has won the most Pulitzer Prizes for journalism and remains the U.S. media's ""newspaper of record"". In 2019, New York City was voted the greatest city in the world in a survey of over 30,000 p...",Pass,YES


#### Benchmark on Generated Question

Now lets generate a few more questions so that we have more to evaluate with and run a small benchmark.

In [25]:
#from llama_index.core.evaluation import DatasetGenerator
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

#question_generator = DatasetGenerator.from_documents(documents)
question_generator = RagDatasetGenerator.from_documents(documents)
eval_questions = question_generator.generate_questions_from_nodes()

In [30]:
eval_questions.to_pandas().head(2)

query  \
0  What are the five boroughs of New York City and which counties do they correspond to?   
1  How did New York City originate and what were its different names throughout history?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [42]:
[e.query for e in eval_questions.examples[0:5]]

['What are the five boroughs of New York City and which counties do they correspond to?',
 'How did New York City originate and what were its different names throughout history?',
 'What factors contribute to New York City being considered a global cultural, financial, and media center?',
 'What factors contribute to New York City being considered the "greatest city in the world" according to a survey of over 30,000 people?',
 'How did New York City get its name, and who was it named in honor of?']

In [33]:
import asyncio

def evaluate_query_engine(query_engine, questions):
    c = [query_engine.aquery(q) for q in questions]
    results = asyncio.run(asyncio.gather(*c))
    print("finished query")

    total_correct = 0
    for r in results:
        # evaluate with gpt 4
        eval_result = (
            1 if evaluator_gpt4.evaluate_response(response=r).passing else 0
        )
        total_correct += eval_result

    return total_correct, len(results)

In [43]:
vector_query_engine = vector_index.as_query_engine()
correct, total = evaluate_query_engine(vector_query_engine, [e.query for e in eval_questions.examples[0:5]]) #eval_questions[:5])

print(f"score: {correct}/{total}")

finished query
score: 5/5


In [ ]:
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio
nest_asyncio.apply()

In [ ]:
def displayify_df(df):
    """For pretty displaying DataFrame in a notebook."""
    display_df = df.style.set_properties(
        **{
            "inline-size": "300px",
            "overflow-wrap": "break-word",
        }
    )
    display(display_df)

In [ ]:
from llama_index.core.llama_dataset import download_llama_dataset
from llama_index.core.llama_pack import download_llama_pack
from llama_index.core import VectorStoreIndex

# download and install dependencies for benchmark dataset
rag_dataset, documents = download_llama_dataset("EvaluatingLlmSurveyPaperDataset", "../Data")

Next, we build a RAG over the same source documents used to created the `rag_dataset`.

In [ ]:
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

With our RAG (i.e `query_engine`) defined, we can make predictions (i.e., generate responses to the query) with it over the `rag_dataset`.

In [ ]:
prediction_dataset = await rag_dataset.amake_predictions_with(predictor=query_engine, batch_size=5, show_progress=True)

In [ ]:
# instantiate the gpt-4 judges
from llama_index.core.evaluation import AnswerRelevancyEvaluator
from llama_index.core.evaluation import ContextRelevancyEvaluator

judges = {}
judges["answer_relevancy"] = AnswerRelevancyEvaluator(llm=OpenAI(temperature=0, model="gpt-3.5-turbo"),)
judges["context_relevancy"] = ContextRelevancyEvaluator(llm=OpenAI(temperature=0, model="gpt-4"),)

Now, we can use our evaluator to make evaluations by looping through all of the <example, prediction> pairs.

In [ ]:
eval_tasks = []
for example, prediction in zip(rag_dataset.examples, prediction_dataset.predictions):
    eval_tasks.append(
        judges["answer_relevancy"].aevaluate(
            query=example.query,
            response=prediction.response,
            sleep_time_in_seconds=1.0,
        )
    )
    eval_tasks.append(
        judges["context_relevancy"].aevaluate(
            query=example.query,
            contexts=prediction.contexts,
            sleep_time_in_seconds=1.0,
        )
    )

### 1.4 Guideline Adherence  
GuidelineEvaluator evaluates a question answer system given user specified guidelines.

In [44]:
from llama_index.core.evaluation import GuidelineEvaluator
import nest_asyncio
nest_asyncio.apply()

In [45]:
GUIDELINES = ["The response should fully answer the query.",
              "The response should avoid being vague or ambiguous.",
              "The response should be specific and use statistics or numbers when possible.",
             ]

evaluators = [GuidelineEvaluator(llm=llm, guidelines=guideline) for guideline in GUIDELINES]

In [46]:
sample_data = {
    "query": "Tell me about global warming.",
    "contexts": [
        (
            "Global warming refers to the long-term increase in Earth's"
            " average surface temperature due to human activities such as the"
            " burning of fossil fuels and deforestation."
        ),
        (
            "It is a major environmental issue with consequences such as"
            " rising sea levels, extreme weather events, and disruptions to"
            " ecosystems."
        ),
        (
            "Efforts to combat global warming include reducing carbon"
            " emissions, transitioning to renewable energy sources, and"
            " promoting sustainable practices."
        ),
    ],
    "response": (
        "Global warming is a critical environmental issue caused by human"
        " activities that lead to a rise in Earth's temperature. It has"
        " various adverse effects on the planet."
    ),
}

In [47]:
for guideline, evaluator in zip(GUIDELINES, evaluators):
    eval_result = evaluator.evaluate(
        query=sample_data["query"],
        contexts=sample_data["contexts"],
        response=sample_data["response"],
    )
    print("=====")
    print(f"Guideline: {guideline}")
    print(f"Pass: {eval_result.passing}")
    print(f"Feedback: {eval_result.feedback}")

=====
Guideline: The response should fully answer the query.
Pass: False
Feedback: The response does not fully answer the query. It briefly mentions that global warming is caused by human activities and has adverse effects, but it lacks depth and detail. More information about the causes, impacts, and potential solutions to global warming should be included to fully address the query.
=====
Guideline: The response should avoid being vague or ambiguous.
Pass: False
Feedback: The response is too vague and lacks specific details about the causes and effects of global warming. It would be helpful to provide more in-depth information to fully address the query.
=====
Guideline: The response should be specific and use statistics or numbers when possible.
Pass: False
Feedback: The response does not provide specific information or use statistics to support the statement about global warming. It would be more effective to include data on temperature increases, greenhouse gas emissions, or other

### 1.5 Question Generation  

We walk through the process of generating a list of questions that could be asked about your data.  
This is useful for setting up an evaluation pipeline using the FaithfulnessEvaluator and RelevancyEvaluator evaluation tools.

In [48]:
#import logging
#import sys
#import pandas as pd

#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [49]:
#from llama_index.core.evaluation import DatasetGenerator, 
from llama_index.core.evaluation import RelevancyEvaluator
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.core.llama_dataset import LabelledRagDataset
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response

In [50]:
reader = SimpleDirectoryReader(input_files=[f"../Data/paul_graham_essay.txt"])
documents = reader.load_data()

In [51]:
data_generator = RagDatasetGenerator.from_documents(documents)

In [52]:
#eval_questions = data_generator.generate_questions_from_nodes()
eval_questions = LabelledRagDataset.from_json("../Data/rag_dataset.json")

In [54]:
eval_questions.to_pandas().head(5)

query  \
0  "Describe the author's early experiences with programming. What was the first machine he used and what challenges did he face while trying to write programs on it?"                                    
1  "What were the author's initial career aspirations before college and how did his experiences in college change his perspective and career path?"                                                       
2  "Discuss the influence of the novel 'The Moon is a Harsh Mistress' and the PBS documentary featuring Terry Winograd on the author's decision to switch to AI. Why did these two sources inspire him?"   
3  "In the context, the author mentions a novel by Heinlein that influenced his interest in AI. What is the name of this novel and how did it inspire the author's interest in AI?"                        
4  "The author discusses his experience with learning Lisp and how it expanded his concept of a program. Can you explain what Lisp is and why it was considered the language of AI during that time?"      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [55]:
[e.query for e in eval_questions.examples[0:5]]

['"Describe the author\'s early experiences with programming. What was the first machine he used and what challenges did he face while trying to write programs on it?"',
 '"What were the author\'s initial career aspirations before college and how did his experiences in college change his perspective and career path?"',
 '"Discuss the influence of the novel \'The Moon is a Harsh Mistress\' and the PBS documentary featuring Terry Winograd on the author\'s decision to switch to AI. Why did these two sources inspire him?"',
 '"In the context, the author mentions a novel by Heinlein that influenced his interest in AI. What is the name of this novel and how did it inspire the author\'s interest in AI?"',
 '"The author discusses his experience with learning Lisp and how it expanded his concept of a program. Can you explain what Lisp is and why it was considered the language of AI during that time?"']

#### Saving our Dataset

In [ ]:
eval_questions.save_json("../Data/rag_dataset.json")

#### Reading the saved Dataset

In [ ]:
eval_questions = LabelledRagDataset.from_json("../Data/rag_dataset.json")

#### Using the Dataset for Evaluation (creating an index with the same source of the questions)

In [56]:
evaluator = RelevancyEvaluator(llm=llm)

In [57]:
# create vector index
vector_index = VectorStoreIndex.from_documents(documents)

In [58]:
# define jupyter display function
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response.response),
            "Source": (response.source_nodes[0].node.get_content()[:1000] + "..."),
            "Evaluation Result": eval_result,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [59]:
eval_questions[1].query

'"What were the author\'s initial career aspirations before college and how did his experiences in college change his perspective and career path?"'

In [76]:
#eval_result.passing
#eval_result.score
#eval_result.feedback

In [68]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[1].query)
eval_result = evaluator.evaluate_response(query=eval_questions[1].query, response=response_vector)
display_eval_df(eval_questions[1].query, response_vector, eval_result.passing)

,Query,Response,Source,Evaluation Result
0,"""What were the author's initial career aspirations before college and how did his experiences in college change his perspective and career path?""","The author's initial career aspirations before college were focused on writing and programming. He wrote short stories and started programming on an IBM 1401 in 9th grade. In college, he initially planned to study philosophy but found it unfulfilling and switched to AI due to his interest sparked by a novel and a PBS documentary. This shift in perspective led him to pursue a career in AI rather than philosophy.","What I Worked On February 2021 Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep. The first programs I tried writing were on the IBM 1401 that our school district used for what was then called ""data processing."" This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights. The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the...",True


In [ ]:
display_eval_df(eval_questions[1].query, response_vector, eval_result.feedback)

### 1.6 Context Relevancy and Answer Relevancy  
`AnswerRelevancyEvaluator` and `ContextRelevancyEvaluator` give a measure on the relevancy of a generated answer and retrieved contexts, respectively, to a given user query. Both of these evaluators return a `score` that is between 0 and 1 as well as a generated `feedback` explaining the score. Note that, higher score means higher relevancy. In particular, we prompt the judge LLM to take a step-by-step approach in providing a relevancy score, asking it to answer the following two questions of a generated answer to a query for answer relevancy (for context relevancy these are slightly adjusted):

1. Does the provided response match the subject matter of the user's query?
2. Does the provided response attempt to address the focus or perspective on the subject matter taken on by the user's query?

Each question is worth 1 point and so a perfect evaluation would yield a score of 2/2.